In [4]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np

In [5]:
def findNameById(id) :
    data_dir = "C:/Users/Admin/jupyter/face-recognition-project/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    id_str = str(id)
    for image in path:
        info = os.path.split(image)[1].split(".")[1]
        lis = info.split("-")
        id_info = lis[0]
        name = lis[1]
        if id_str == id_info:
            return name

In [6]:
window = tk.Tk()

window.title("Face recognition system")

l1 = tk.Label(window,text="Name", font=("Algerian", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window,text="Age", font=("Algerian", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window,text="Identity", font=("Algerian", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

def train_classifier() :
    data_dir = "C:/Users/Admin/jupyter/face-recognition-project/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id_str = os.path.split(image)[1].split(".")[1]
        id = int(id_str.split("-")[0])
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    
    #Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result", "Training dataset completed")

b1 = tk.Button(window, text="Training", font=("Algerian", 20), bg="orange", fg="red", command=train_classifier)
b1.grid(column=0, row=4)

def detect_face() :
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf) :
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

        coords = []
        for(x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            id, pred = clf.predict(gray_image[y:y+h,x:x+w])
            print(id)
            print("Pred: " + str(pred))
            confidence = int(100 * (1 - pred/300))
            print("confidence: " + str(confidence))
            if confidence >= 85 :
                cv2.putText(img, findNameById(abs(id)), (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else :
                cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

            coords = [x, y, w, h]
        return coords

    def recognize(img, clf, faceScascade) :
        coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade)
        cv2.imshow("face detection", img)

        if cv2.waitKey(1) == 13 :
            break
    video_capture.release()
    cv2.destroyAllWindows()

b2 = tk.Button(window, text="Detect the face", font=("Algerian", 20), bg="green", fg="white", command=detect_face)
b2.grid(column=1, row=4)


def generate_dataset() :
    if(t1.get() == "" or t2.get() == "" or t3.get() == "") :
        messagebox.showinfo("Result", "Please provide complete details of the user")
    else :
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            #scaling factor=1.3
            #Minimum neighbor=5
            if faces is () :
                return None
            for(x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
            return cropped_face
        cap = cv2.VideoCapture(0)
        id = t3.get() + "-" + t1.get()
        img_id = 0

        while True :
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id += 1
                face = cv2.resize(face_cropped(frame), (400, 400))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user." + id + "." + str(img_id) + ".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

                cv2.imshow("Cropped face", face)
                if cv2.waitKey(1) == 13 or int(img_id) == 100:
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result', 'Generating dataset completed')

b3 = tk.Button(window, text="Generate Dataset", font=("Algerian", 20), bg="pink", fg="black", command=generate_dataset)
b3.grid(column=2, row=4)

window.geometry("800x200")
window.mainloop()

<>:99: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:99: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Admin\AppData\Local\Temp\ipykernel_10488\3244880125.py:99: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is () :


123456
Pred: 43.914489268199944
confidence: 85
123456
Pred: 38.26875359020394
confidence: 87
123456
Pred: 38.26875359020394
confidence: 87
123456
Pred: 37.59088141729627
confidence: 87
123456
Pred: 38.76098557317638
confidence: 87
123456
Pred: 38.76098557317638
confidence: 87
123456
Pred: 39.38910644618378
confidence: 86
123456
Pred: 40.010550026245475
confidence: 86
123456
Pred: 38.295584647413094
confidence: 87
123456
Pred: 38.295584647413094
confidence: 87
123456
Pred: 36.85089694425747
confidence: 87
123456
Pred: 36.95727688053905
confidence: 87
123456
Pred: 36.95727688053905
confidence: 87
123456
Pred: 37.315501061052906
confidence: 87
123456
Pred: 39.063526834782195
confidence: 86
123456
Pred: 39.063526834782195
confidence: 86
123456
Pred: 40.919631113755365
confidence: 86
123456
Pred: 44.75121078739134
confidence: 85
123456
Pred: 44.75121078739134
confidence: 85
123456
Pred: 43.28838115857449
confidence: 85
123456
Pred: 41.1324863353682
confidence: 86
123456
Pred: 41.13248633536

123456
Pred: 42.99943541456994
confidence: 85
123456
Pred: 42.55334481504103
confidence: 85
123456
Pred: 42.55334481504103
confidence: 85
123456
Pred: 45.68743084157514
confidence: 84
123456
Pred: 45.68743084157514
confidence: 84
123456
Pred: 44.380938101349
confidence: 85
123456
Pred: 40.393193873996715
confidence: 86
123456
Pred: 40.393193873996715
confidence: 86
123456
Pred: 40.393193873996715
confidence: 86
123456
Pred: 41.75928571061568
confidence: 86
123456
Pred: 41.75928571061568
confidence: 86
123456
Pred: 42.24951654448788
confidence: 85
123456
Pred: 42.24951654448788
confidence: 85
123456
Pred: 44.72832431473264
confidence: 85
123456
Pred: 44.72832431473264
confidence: 85
123456
Pred: 44.83654574763536
confidence: 85
123456
Pred: 44.83654574763536
confidence: 85
123456
Pred: 44.758969508440764
confidence: 85
123456
Pred: 44.758969508440764
confidence: 85
123456
Pred: 43.90077295321632
confidence: 85
123456
Pred: 43.90077295321632
confidence: 85
123456
Pred: 44.268590722081925